<a href="https://colab.research.google.com/github/AtharvaTijare/Artificial-Intelligence/blob/main/p1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# prompt: 1b. Implementation of Block World Problem using Planning Technique in python

from collections import deque

# Define the initial state and goal state
# A state is represented as a dictionary where keys are blocks
# and values are the block below it ('table' or another block).
# 'clear' indicates if the block has nothing on top.

initial_state = {
    'A': 'table', 'B': 'table', 'C': 'table',
    'clear_A': True, 'clear_B': True, 'clear_C': True
}

goal_state = {
    'A': 'B', 'B': 'C', 'C': 'table',
    'clear_A': True, 'clear_B': True, 'clear_C': True
}

# Define possible actions
# An action is a tuple: (action_name, block_to_move, destination)

def is_clear(state, block):
    """Checks if a block is clear in the given state."""
    if block == 'table':
        return True # The table is always clear
    return state.get(f'clear_{block}', False) # Use .get to avoid KeyError if clear_block doesn't exist for some reason


def is_on(state, block, destination):
    """Checks if a block is on the specified destination in the given state."""
    return state[block] == destination

def apply_action(state, action):
    """Adds or removes a block from a stack based on the provided action."""
    new_state = state.copy()
    action_name, block_to_move, destination = action

    if action_name == 'move_to_table':
        # Preconditions: block_to_move is clear and is on some other block
        if is_clear(state, block_to_move) and state[block_to_move] != 'table':
            block_below = state[block_to_move]
            new_state[block_to_move] = 'table'
            new_state[f'clear_{block_below}'] = True # The block below is now clear
            new_state[f'clear_{block_to_move}'] = True # The moving block remains clear
            return new_state
    elif action_name == 'move_block':
        # Preconditions: block_to_move is clear, destination is clear and destination is not the block being moved
        if is_clear(state, block_to_move) and is_clear(state, destination) and block_to_move != destination:
            block_below = state[block_to_move]
            new_state[block_to_move] = destination
            new_state[f'clear_{block_to_move}'] = True # The moving block remains clear
            # The block it was on is now clear, unless it was on the table
            if block_below != 'table':
                new_state[f'clear_{block_below}'] = True
            new_state[f'clear_{destination}'] = False # The destination block is no longer clear
            return new_state
    return None # Action not applicable

def get_possible_actions(state):
    """Returns a list of possible actions from the current state."""
    actions = []
    blocks = [block for block in state if not block.startswith('clear_')]
    for block in blocks:
        if is_clear(state, block):
            # Possible to move this clear block
            if state[block] != 'table':
                # Can move to table if it's on another block
                actions.append(('move_to_table', block, 'table'))
            for dest in blocks + ['table']:
                if dest != block:
                    # Can move to another clear block or the table
                    if is_clear(state, dest): # Check if destination is clear
                         # Ensure not trying to move onto itself
                         if state[block] != dest: # Prevent moving onto the same block it's already on
                            actions.append(('move_block', block, dest))
    return actions


def are_states_equal(state1, state2):
    """Compares two states for equality."""
    # We only need to compare the block positions, not the clear status,
    # as the clear status is derived from block positions.
    blocks1 = {k: v for k, v in state1.items() if not k.startswith('clear_')}
    blocks2 = {k: v for k, v in state2.items() if not k.startswith('clear_')}
    return blocks1 == blocks2


# Breadth-First Search for planning
def solve_block_world(initial_state, goal_state):
    """Solves the Block World problem using BFS."""
    queue = deque([(initial_state, [])]) # (current_state, path_of_actions)
    visited = set()

    while queue:
        current_state, current_path = queue.popleft()

        # Check if current state is the goal state
        if are_states_equal(current_state, goal_state):
            return current_path # Found a solution

        # Avoid visiting the same state again
        state_tuple = tuple(sorted(current_state.items())) # Hashable representation of state
        if state_tuple in visited:
            continue
        visited.add(state_tuple)

        # Explore possible actions from the current state
        for action in get_possible_actions(current_state):
            next_state = apply_action(current_state, action)
            if next_state:
                queue.append((next_state, current_path + [action]))

    return None # No solution found

# Run the solver
solution = solve_block_world(initial_state, goal_state)

if solution:
    print("Solution found:")
    for i, action in enumerate(solution):
        print(f"Step {i+1}: {action}")
else:
    print("No solution found.")

Solution found:
Step 1: ('move_block', 'B', 'C')
Step 2: ('move_block', 'A', 'B')


In [6]:
# Here’s a **clear and detailed pseudocode (logic explanation)** of your Python implementation of the **Block World Problem using Planning Technique (BFS)**:

# ---

# ## 🔍 **Pseudocode / Logic Explanation**

# ---

# ### 1. **Define Initial State and Goal State**

# * Use a dictionary to represent the **current position of blocks**.
# * Include extra information like `clear_X = True/False` to know which blocks are free to move.

# ```pseudocode
# initial_state = {
#     'A': 'table', 'B': 'table', 'C': 'table',
#     'clear_A': True, 'clear_B': True, 'clear_C': True
# }

# goal_state = {
#     'A': 'B', 'B': 'C', 'C': 'table',
#     'clear_A': True, 'clear_B': True, 'clear_C': True
# }
# ```

# ---

# ### 2. **Define Helper Functions**

# #### a. `is_clear(state, block)`

# * Returns `True` if the block is clear (nothing on top).
# * Table is always considered clear.

# #### b. `is_on(state, block, destination)`

# * Checks if the block is on the specified destination.

# ---

# ### 3. **`apply_action(state, action)`**

# * Takes a current state and a move (action), and returns the **new state**.
# * Two types of moves:

#   * **Move block to table**: if it’s clear and on another block.
#   * **Move block onto another block**: if both blocks are clear.

# ---

# ### 4. **`get_possible_actions(state)`**

# * Generates a list of all valid actions from the current state:

#   * For every block that is clear:

#     * Try moving it to the table (if it’s not already there).
#     * Try placing it on any other clear block (except itself and where it already is).

# ---

# ### 5. **`are_states_equal(state1, state2)`**

# * Compares only block positions (ignores `clear_X` flags).
# * Helps check whether current state matches goal state.

# ---

# ### 6. **`solve_block_world(initial_state, goal_state)`**

# * Use **Breadth-First Search (BFS)**:

# ```pseudocode
# 1. Initialize queue with (initial_state, empty path)
# 2. Initialize an empty set for visited states

# 3. WHILE queue is not empty:
#     a. Pop the first item from queue → (current_state, current_path)

#     b. IF current_state equals goal_state:
#         RETURN current_path as the solution

#     c. Convert current_state to a hashable form (tuple) and add to visited

#     d. FOR each possible action from current_state:
#         i. Apply the action to get next_state
#         ii. IF next_state is valid:
#             - Add (next_state, current_path + [action]) to queue

# 4. RETURN None if no solution is found
# ```

# ---

# ### 7. **Run the Solver and Display Result**

# ```pseudocode
# solution = solve_block_world(initial_state, goal_state)

# IF solution exists:
#     Print each step of the solution
# ELSE:
#     Print "No solution found"
# ```

# ---

# ## ✅ Example Output

# ```
# Solution found:
# Step 1: ('move_block', 'A', 'B')
# Step 2: ('move_block', 'B', 'C')
# ```

# ---

# ## 🔁 Summary of the Process

# * Model block relationships using a dictionary.
# * Generate possible legal moves.
# * Apply moves and use **BFS** to find the shortest sequence from start to goal.
# * Return the path of actions once the goal is reached.

# ---

# Would you like a **flowchart or diagram** for better understanding as well?


SyntaxError: invalid character '’' (U+2019) (<ipython-input-6-3282058509>, line 1)

In [ ]:
# 🧠 Theory: Block World Problem Using Planning Technique (BFS)
# 📌 Introduction
# The Block World Problem is a classic AI planning problem used to test the capabilities of search algorithms and state-space representation. It involves a set of blocks placed on a table, and the goal is to rearrange them to match a desired configuration using a series of legal actions.

# 🎯 Problem Description
# You are given blocks labeled (e.g., A, B, C).

# Each block can be on the table or on top of another block.

# A block is "clear" if there is nothing on top of it.

# The objective is to move blocks from an initial state to a goal state by applying valid actions.

# 🧱 State Representation
# Each state is represented as a dictionary:

# The position of each block (block: on_what), e.g., 'A': 'table' or 'A': 'B'.

# The clear status of each block: e.g., 'clear_A': True if nothing is on A.

# Example:
# python
# Copy
# Edit
# {
#     'A': 'B',     # A is on B
#     'B': 'C',     # B is on C
#     'C': 'table', # C is on the table
#     'clear_A': True,
#     'clear_B': False,
#     'clear_C': False
# }
# ⚙️ Actions (Operators)
# Two types of valid moves are defined:

# Move block to table:
# Only allowed if the block is clear and currently on another block.

# Move block onto another block:
# Allowed if both blocks are clear, and they are not the same block.

# Each action updates the state by:

# Changing the position of the block.

# Updating the clear status of the involved blocks.

# 🔁 Planning Technique Used: BFS (Breadth-First Search)
# The algorithm explores all possible sequences of actions level by level to find the shortest path from the initial to the goal state.

# BFS Steps:
# Start from the initial state.

# Generate all valid actions from the current state.

# For each action, apply it to generate a new state.

# Use a queue to explore each path systematically.

# Keep track of visited states to avoid loops.

# Stop when the goal state is reached.

# 🔎 Goal Test
# A goal is reached when the position of all blocks in the current state matches the positions in the goal state. The clear status is automatically maintained based on block positions and need not be compared directly.

# 🧪 Advantages of BFS in Block World
# Guarantees the shortest solution path (minimum number of moves).

# Explores all possibilities systematically.

# Suitable for small problem sizes.

# ⚠️ Limitations
# Can be memory-intensive due to storing all explored states.

# Performance drops for large numbers of blocks due to state-space explosion.